In [ ]:
data_link = 's3://nisar-st-data-ondemand/ALOS-1-data/L0B/ALPSRP267700710-L0B.h5' # string
gpu_enabled = 1 # boolean
config_yml = '' # string
timestamp = '20240131T0123456' # string

# hysds specifications
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = '10GB'
_submission_type = 'iteration'
_label = 'STATIC LAYER PGE'

In [ ]:
# Convert boolean parameters because they must be specified as strings
try:
    if not isinstance(gpu_enabled, bool):
        gpu_enabled = int(gpu_enabled) > 0
except ValueError:
    if instance(gpu_enabled, str):
        gpu_enabled = gpu_enabled.lower() == 'true'
    else:
        gpu_enabled = False
print(f'{gpu_enabled=}')

In [ ]:
import os
import yaml
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PRODUCT_DIR, exist_ok=True)

if config_yml == '':
    with open(os.path.join(ISCE3_BUILD_DIR, 'share', 'nisar', 'defaults', 'static.yaml'), 'r') as f:
        CONFIG_YML = yaml.safe_load(f)
else:
    print('Using custom static.py run config...')
    CONFIG_YML = yaml.safe_load(config_yml)

yml_path = os.path.join(OUTPUT_DIR, 'static_final.yaml')

print(WORKING_DIR)

In [ ]:
import isce3_config

isce3_config.write_static_config(CONFIG_YML, yml_path)

!python -m nisar.workflows.static ${yml_path}

In [ ]:
import h5py
import json

# Grab some appropriate values for PCM to automatically stage out
# try:
#     h5_file = h5py.File(output_f, 'r')
#     polarization = h5_file['science']['LSAR']['RSLC']['swaths']['frequencyA']['listOfPolarizations'][0].decode('utf-8')
# except Exception as e:
#     print(f'Exception occured: {e}')
#     print(f'As the polarization was unable to be automatically detected, the polarization will be listed as \'HH\' by default.')
polarization = 'HH'
regex_name = isce3_config.RSLC_FORMAT.format(
    polarization=f'{polarization}00',
    timestamp=timestamp)
auto_output_dir = os.path.join(WORKING_DIR, regex_name)

# Create the automatic stage-out directory
!mkdir -p {auto_output_dir}
!cp {WORKING_DIR}/*-output.ipynb {auto_output_dir}
!cp {WORKING_DIR}/*.txt {auto_output_dir}
!cp {WORKING_DIR}/*.json {auto_output_dir}
!mv {yml_path} {auto_output_dir}
#!mv {output_f} {auto_output_dir}

# Move the new path variables to avoid confusion
yml_path = os.path.join(auto_output_dir, os.path.basename(yml_path))
output_f = os.path.join(auto_output_dir, os.path.basename(output_f))

# Create metadata files for the automatic stage-out directory
with open(os.path.join(auto_output_dir, f'{regex_name}.met.json'), 'w', encoding='utf-8') as f:
    content = {
        'polarization': polarization,
    }
    f.write(json.dumps(content))
with open(os.path.join(auto_output_dir, f'{regex_name}.dataset.json'), 'w', encoding='utf-8') as f:
    content = {
         'version': 'v1.0',
         'label': 'This is purely an EXAMPLE metadata file, the values in this file are not representative of this product.',
         'location': {
           'type': 'polygon',
           'coordinates': [
             [
                [-122.9059682940358,40.47090915967475],
                [-121.6679748715316,37.84406528996276],
                [-120.7310161872557,38.28728069813177],
                [-121.7043611684245,39.94137004454238],
                [-121.9536916840953,40.67097860759095],
                [-122.3100379696548,40.7267890636145],
                [-122.7640648263371,40.5457010812299],
                [-122.9059682940358,40.47090915967475]
              ]
            ]
        },
        'starttime': '2017-01-01T00:00:00',
        'endtime': '2017-01-01T00:05:00',
    }
    f.write(json.dumps(content))